In [14]:
from transformers import BertForTokenClassification, BertTokenizer, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import numpy as np
import torch
import torch
from torch import nn
import ast
from transformers import BertModel
from sklearn.metrics import accuracy_score, f1_score


In [15]:
class DocNerBERT(nn.Module):
    def __init__(self, num_doc_labels, num_token_labels):
        super().__init__()
        
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(0.1)

        self.doc_classifier = nn.Linear(self.bert.config.hidden_size, num_doc_labels)
        self.token_classifier = nn.Linear(self.bert.config.hidden_size, num_token_labels)

        self.loss = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, doc_labels=None, token_labels=None):
        outputs = self.bert(input_ids, attention_mask = attention_mask)

        # Document classification
        doc_output = outputs.pooler_output
        doc_output = self.dropout(doc_output)
        doc_logits = self.doc_classifier(doc_output)
        
        # Token classification
        token_output = outputs.last_hidden_state
        token_output = self.dropout(token_output)
        token_logits = self.token_classifier(token_output)

        if doc_labels is not None and token_labels is not None:
            doc_loss = self.loss(doc_logits, doc_labels)
            token_loss = self.loss(token_logits.view(-1, token_logits.shape[-1]), token_labels.view(-1))
            loss = doc_loss + token_loss

        return {
            'loss': loss,
            'doc_logits': doc_logits,
            'token_logits': token_logits
        }


In [16]:
def prepare_data(df):
    texts = df['tokens'].apply(lambda x: ' '.join(ast.literal_eval(x))).tolist()
    doc_labels = df['sentence_label'].tolist()
    token_labels = df['ner_tags'].apply(ast.literal_eval).tolist()
    
    return {
        "text": texts,
        "doc_labels": doc_labels,
        "token_labels": token_labels
    }

def tokenize_and_align_labels(examples, tokenizer, max_length=128):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_length)
    
    labels = []
    for i, label in enumerate(examples["token_labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx] if word_idx < len(label) else -100)
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    
    tokenized_inputs["token_labels"] = labels
    tokenized_inputs["doc_labels"] = examples["doc_labels"]
    return tokenized_inputs

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    doc_logits, token_logits = logits
    doc_labels, token_labels = labels

    doc_predictions = np.argmax(doc_logits, axis=-1)
    token_predictions = np.argmax(token_logits, axis=-1)

    # Compute document classification accuracy
    doc_correct = (doc_predictions == doc_labels).sum()
    doc_total = len(doc_labels)
    doc_accuracy = doc_correct / doc_total

    # Compute token classification accuracy (ignoring padding tokens)
    token_correct = ((token_predictions == token_labels) & (token_labels != -100)).sum()
    token_total = (token_labels != -100).sum()
    token_accuracy = token_correct / token_total

    return {
        "doc_accuracy": doc_accuracy,
        "token_accuracy": token_accuracy
    }


In [17]:
# Read CSV file
df = pd.read_csv('train_augmented.csv')

# Prepare data
data = prepare_data(df)


In [18]:

# Initialize tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


In [19]:

# Create a Dataset
dataset = Dataset.from_dict(data)
tokenized_dataset = dataset.map(
    lambda examples: tokenize_and_align_labels(examples, tokenizer),
    batched=True,
    remove_columns=dataset.column_names
)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 4919.91 examples/s]


In [20]:


# Initialize model
num_doc_labels = len(df['sentence_label'].unique())
num_token_labels = max(max(ast.literal_eval(x)) for x in df['ner_tags']) + 1
model = DocNerBERT(num_doc_labels, num_token_labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

In [21]:


# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Save the model
trainer.save_model("./docnerbert_model")

  3%|▎         | 10/375 [01:18<44:00,  7.23s/it] 

{'loss': 3.8495, 'grad_norm': 12.97952651977539, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.08}


  5%|▌         | 20/375 [02:27<40:58,  6.92s/it]

{'loss': 3.7017, 'grad_norm': 11.461528778076172, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.16}


  8%|▊         | 30/375 [03:37<39:56,  6.95s/it]

...

{'loss': 1.1017, 'grad_norm': 16.127687454223633, 'learning_rate': 3.6e-05, 'epoch': 2.88}


 99%|█████████▊| 370/375 [43:59<00:34,  6.91s/it]

{'loss': 0.9666, 'grad_norm': 17.190067291259766, 'learning_rate': 3.7e-05, 'epoch': 2.96}


100%|██████████| 375/375 [44:33<00:00,  7.13s/it]


{'train_runtime': 2673.9341, 'train_samples_per_second': 1.122, 'train_steps_per_second': 0.14, 'train_loss': 1.6900064697265624, 'epoch': 3.0}
